## (4.1) MNI Atlas registrations, transformations and segmentations using Tissue models, Label propagartion and Bayesian Model. 

In [1]:
# Import Libraries
import os
from pathlib import Path
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import utils
import pickle

In [2]:
# Get the current working directory
cwd = os.getcwd()

In [3]:
atlas_name ="MNI"
# Load list of patient ID's
list_patient = utils.read_patients_list(cwd + "/data/testing-set/testing-images")

# Register MNI atlas to the test image space to obtain label propagation information

# Load the MNI atlas
atlas = sitk.ReadImage(cwd + f"/data/{atlas_name}_atlas/atlas.nii.gz")
# Load the MNI atlas template
atlas_template = sitk.ReadImage(cwd + f"/data/{atlas_name}_atlas/template.nii.gz")

atlas_array = sitk.GetArrayFromImage(atlas)
atlas_array.shape
atlas_template_array = sitk.GetArrayFromImage(atlas_template)
atlas_template_array.shape

(182, 218, 182)

In [4]:
# Divide the atlas into 4 
# atlas_array_1 = atlas_array[0, :, :]
# atlas_array_2 = atlas_array[1, :, :]
# atlas_array_3 = atlas_array[2, :, :]
# atlas_array_4 = atlas_array[3, :, :]

# # save the atlas array as nifti image
# utils.save_as_nifti(atlas_array_1, cwd + f"/data/{atlas_name}_atlas/atlas_background.nii.gz", cwd + f"/data/{atlas_name}_atlas/template.nii.gz")
# utils.save_as_nifti(atlas_array_2, cwd + f"/data/{atlas_name}_atlas/atlas_csf.nii.gz", cwd + f"/data/{atlas_name}_atlas/template.nii.gz")
# utils.save_as_nifti(atlas_array_3, cwd + f"/data/{atlas_name}_atlas/atlas_gm.nii.gz", cwd + f"/data/{atlas_name}_atlas/template.nii.gz")
# utils.save_as_nifti(atlas_array_4, cwd + f"/data/{atlas_name}_atlas/atlas_wm.nii.gz", cwd + f"/data/{atlas_name}_atlas/template.nii.gz")

In [5]:
def resize_images(fixed_path, moving_path, p, name):
    template = sitk.ReadImage(moving_path)
    template_array = sitk.GetArrayFromImage(template)

    # Read the test image
    test_image = sitk.ReadImage(fixed_path)
    test_image_array = sitk.GetArrayFromImage(test_image)

    # Reshape the template image to the same size as the test image
    template_array_resized = np.zeros(test_image_array.shape)
    template_array_resized[0:template_array.shape[0], 0:template_array.shape[1], 0:template_array.shape[2]] = template_array
    
    # Save the resized template image
    utils.save_as_nifti(template_array_resized, cwd + f"/data/MNI_atlas/{name}_resized_{p}.nii.gz", fixed_path)

    # Read the resized template image
    template_resized = str(cwd) + f"/data/MNI_atlas/{name}_resized_{p}.nii.gz"

    return template_resized

In [6]:
# Register the template to the test image space

# Read atlasses
atlas_background = cwd + f"/data/{atlas_name}_atlas/atlas_background.nii.gz"
atlas_path_csf = cwd + f"/data/{atlas_name}_atlas/atlas_csf.nii.gz"
atlas_path_gm = cwd + f"/data/{atlas_name}_atlas/atlas_gm.nii.gz"
atlas_path_wm = cwd + f"/data/{atlas_name}_atlas/atlas_wm.nii.gz"

# Go through all the patients in the test-set
for patient in list_patient[8:]:
    print(patient)
    # Read the images
    fixed_path = str(cwd) + f'/data/testing-set/testing-images/{patient}.nii.gz'
    labels_path = str(cwd) + f'/data/testing-set/testing-labels/{patient}_3C.nii.gz'
    moving_path = str(cwd) + f'/data/MNI_atlas/template.nii.gz'

    # Apply resizing to the images
    template_resized_path = resize_images(fixed_path, moving_path, patient, name="template")

    # Register the images
    resultImage, transformParameterMap = utils.image_registration(fixed_path, template_resized_path)

    atlas_path_csf1 = resize_images(fixed_path, atlas_path_csf, patient, name="atlas_csf")
    atlas_path_gm1 = resize_images(fixed_path, atlas_path_gm, patient, name="atlas_gm")
    atlas_path_wm1 = resize_images(fixed_path, atlas_path_wm, patient, name="atlas_wm")

    # Transform the atlas images
    atlas_csf_registered, atlas_gm_registered, atlas_wm_registered = utils.transform_atlas(atlas_path_csf1, atlas_path_gm1, atlas_path_wm1, transformParameterMap)
    # Save the transformed atlas images
    utils.save_as_nifti(sitk.GetArrayFromImage(atlas_csf_registered), f'data/MNI_atlas/transformed_atlas/csf_atlas_{patient}.nii.gz', fixed_path)
    utils.save_as_nifti(sitk.GetArrayFromImage(atlas_gm_registered), f'data/MNI_atlas/transformed_atlas/gm_atlas_{patient}.nii.gz', fixed_path)
    utils.save_as_nifti(sitk.GetArrayFromImage(atlas_wm_registered), f'data/MNI_atlas/transformed_atlas/wm_atlas_{patient}.nii.gz', fixed_path)


1038
1039
1101
1104
1107
1110
1113
1116
1119
1122
1125
1128
